### Import Libraries

In [1]:
from wrapper_functions_CAR import *
tf.config.run_functions_eagerly(True)
np.random.seed(10)

C:\Users\Admin-Dell\anaconda3\lib\site-packages\gpflow\experimental\utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(
C:\Users\Admin-Dell\anaconda3\lib\site-packages\gpflow\experimental\utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(


TensorFlow version: 2.11.0. Expected: 2.7.0
TensorFlow Probability version: 0.19.0. Expected: 0.15.0


### Create the fake data



In [11]:
training2010 = pd.read_csv('../data/merged_wp_census_data2_081122.csv')
county_adj = pd.read_csv('../data/countyadj2.csv', index_col = 0)
#models = ['acs', 'pep', 'worldpop']
models = ['acs', 'pep']

In [12]:
data_NY, adj_NY = subset_data_by_state(training2010, county_adj, 'New York', 'NY')
#phi_true, u_true, data = simulate_data(data_NY, adj_NY, pivot = -1, sim_numbers = True, one_model = False, models = models)

# Default Configs

In [ ]:
# MCMC configs.
mcmc_step_size=0.1 # @param
mcmc_sample_size=500 # @param
mcmc_num_steps=10_000 # @param
mcmc_burnin=2_500 # @param
mcmc_nchain=10 # @param
mcmc_seed=0 # @param

DEFAULT_MCMC_CONFIG = dict(step_size=mcmc_step_size, 
                           num_steps=mcmc_sample_size, 
                           burnin=mcmc_burnin, 
                           nchain=mcmc_nchain, 
                           seed=mcmc_seed)

#### Running with HMC

In [ ]:
mcmc_config = DEFAULT_MCMC_CONFIG.copy()
mcmc_config.update(dict(burnin = 2_500, num_steps = 10_000, nchain = 5, 
                        kernel_type = 'hmc', step_adaptor_type = 'simple'))
mcmc_config

In [13]:
res_dict = {}
for pivot_DGP in range(-1, 2):
    print('pivot DGP: ' + str(pivot_DGP))
    
    phi_true, u_true, data_pivot = simulate_data(data_NY[:], 
                                                 adj_NY[:], 
                                                 sim_numbers = False,
                                                 scale_down = 1000,
                                                 poisson_noise = True,
                                                 pivot = pivot_DGP, 
                                                 one_model = False, 
                                                 models = models)
    
    for pivot_fit in range(-1, 2):
        print('pivot fit: ' + str(pivot_fit))
        t0 = time.perf_counter()
        CAR_samples, chain_samples, sampler_stat = run_mcmc_CAR(data = data_pivot[:],
                                                                adjacency = adj_NY[:],
                                                                pivot = pivot_fit,
                                                                models = models,                                                        
                                                                debug_mode = True,
                                                                **mcmc_config)  
        print(time.perf_counter() - t0)
        
        res_dict['pDGP: ' + str(pivot_DGP) + '; pfit: ' + str(pivot_fit)] = [CAR_samples, chain_samples, sampler_stat, phi_true, u_true, data_pivot, pivot_DGP, pivot_fit, models]
        

Saving and loading Python objects with Pickle

In [ ]:
data_pivot

In [ ]:
import pickle

# using local data file because these are too big for github
#local_dir = 'C:/Users/nickl/Dropbox/Nick_Cranston/HSPH/Research/Nethery Project/Data/'
local_dir = 'C:/Users/Admin-Dell/Dropbox/Nick_Cranston/HSPH/Research/Nethery Project/Data/'
with open(local_dir + 'CAR_samples_NY_n10000_realdata_div1000_2models_May152023.pickle', 'wb') as results_file:
  pickle.dump([res_dict, mcmc_config], results_file)
  #pickle.dump([CAR_samples, CAR_samples0, CAR_samples1, CAR_samples2, CAR_samples], results_file)

### Loading results and analyzing

In [17]:
local_dir = 'C:/Users/Admin-Dell/Dropbox/Nick_Cranston/HSPH/Research/Nethery Project/Data/'
file = local_dir + 'CAR_samples_NY_n10000_realdata_2models_May152023.pickle'
with open(file, "rb") as input_file:
     #CAR_samples, chain_samples, sampler_stat, mcmc_config, phi_true, u_true, data_sub = pickle.load(input_file)
    res_dict, mcmc_config = pickle.load(input_file)

In [ ]:
data_pivot[['census','acs','pep']]

## Getting the rhat and ESS values

In [ ]:
for key in res_dict.keys():
    sampler_stat = res_dict[key][2]
    stepsize_vec.append(sampler_stat[0][0].numpy())
    
# make the data frame
df_stepsize = pd.DataFrame(np.reshape(np.array(stepsize_vec), (4,4)), 
                       index = ['none','acs','pep','WP'], 
                       columns = ['none','acs','pep','WP'])
df_stepsize

In [45]:
## Get the convergence diagnostics from model fits
rhat_vec = []
for key in res_dict.keys():
    #print(key)
    chain_samples = res_dict[key][1]
    u_samples = phi_to_u(chain_samples, res_dict[key][7])
    ESS = tfp.mcmc.effective_sample_size(u_samples, cross_chain_dims = 1).numpy()
    rhat = tfp.mcmc.potential_scale_reduction(chain_samples, independent_chain_ndims=1).numpy()
    print(np.quantile(ESS, [0.5, 0.025, 0.975]))
    print(np.quantile(rhat, [0.5, 0.025, 0.975]))

,none,acs,pep,WP
none,1.571394,9.714918e+08,9.803074e+02,1558.569214
acs,7.384605,3.776405e+04,7.028971e+07,5.132687
pep,10651.396484,3.741119e+00,1.448885e+05,3.363313
WP,2.530257,4.024219e+00,4.721138e+01,125924.406250


## Plotting the density of ensemble weights

In [ ]:
import seaborn as sns
#plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['figure.figsize'] = [15, 3]
labels = models[:]
labels.insert(0, 'none')

fig = plt.figure()
iter = 0

for pivot_DGP in range(-1,0):
    
    iter = iter + 1
    
    u_true = res_dict['pDGP: ' + str(pivot_DGP) + '; pfit: -1'][4]
    
    CAR_df = pd.DataFrame(u_true[0,:,:], columns = models)
    #CAR_df = pd.DataFrame(CAR_ensemble_weights.numpy(), columns = ["acs","pep","worldpop"])

    #plt.subplot(3, 4, iter)
    plt.subplot(1, 4, iter)
    sns.set(style="darkgrid")
    tt = sns.kdeplot(CAR_df['acs'], shade=True, color="r", label = 'acs')
    tt = sns.kdeplot(CAR_df["pep"], shade=True, color="b", label = 'pep')
    #tt = sns.kdeplot(CAR_df["worldpop"], shade=True, color="g", label = 'worldpop')
    if pivot_DGP == -1:
        plt.title('u true')
    plt.ylabel(labels[pivot_DGP + 1])
    plt.xlabel('')
    plt.xlim(0,1)
    
    for pivot_fit in range(-1,2):
        iter = iter + 1
        key = 'pDGP: ' + str(pivot_DGP) + '; pfit: ' + str(pivot_fit)  

        # get the ensemble weights
        CAR_ensemble_phis = tf.reduce_mean(res_dict[key][0][0], axis = 2).numpy()
        u = phi_to_u(CAR_ensemble_phis, pivot = pivot_fit)
        CAR_df = pd.DataFrame(u, columns = models)

        #plt.subplot(3, 4, iter)
        plt.subplot(1, 4, iter)
        sns.set(style="darkgrid")
        tt = sns.kdeplot(CAR_df['acs'], shade=True, color="r", label = 'acs')
        tt = sns.kdeplot(CAR_df["pep"], shade=True, color="b", label = 'pep')
        #tt = sns.kdeplot(CAR_df["worldpop"], shade=True, color="g", label = 'worldpop')
        #plt.legend()
        plt.xlabel('')
        plt.ylabel('')
        if pivot_DGP == -1:
            plt.title(labels[pivot_fit + 1])
        plt.xlim(0,1)
        
plt.legend()
plt.show()

## Plotting scatterplots of the fitted vs true u values

In [ ]:
key = 'pDGP: -1; pfit: -1'
phis = res_dict[key][0][0]

In [ ]:
plt.rcParams['figure.figsize'] = [12, 3]
labels = models[:]
labels.insert(0, 'none')

fig = plt.figure()
iter = 0

for pivot_DGP in range(-1,0):
    
    ## Get the true u values
    u_true = res_dict['pDGP: ' + str(pivot_DGP) + '; pfit: -1'][4]
    CAR_df = pd.DataFrame(u_true[0,:,:], columns = models)
    
    for pivot_fit in range(-1,2):
        iter = iter + 1
        key = 'pDGP: ' + str(pivot_DGP) + '; pfit: ' + str(pivot_fit)    

        ## get the ensemble weights
        phi = res_dict[key][0][0]
        
        ## Get the u values from each run
        u = np.empty(shape=(phi.shape[0], 
                              len(models), 
                              phi.shape[2]), 
                       dtype='float64')
        for i in range(phi.shape[2]):
            u[:,:,i] = phi_to_u(phi.numpy()[:,:,i], pivot = pivot_fit)   
            
        CAR_df = pd.DataFrame(np.mean(u, axis = 2), columns = models)

        #plt.subplot(3, 4, iter)
        plt.subplot(1, 3, iter)

        x = np.ndarray.flatten(u_true[0,:,:].numpy())
        y = np.ndarray.flatten(CAR_df.values)
        plt.scatter(x, y)
        plt.xlabel("true u")
        plt.ylabel(labels[pivot_fit + 1])

        plt.xlim([min([min(x), min(y)]), max([max(x), max(y)])])
        plt.ylim([min([min(x), min(y)]), max([max(x), max(y)])])
        ax = plt.gca()
        ax.set_aspect('equal', adjustable='box')

        ## obtain m (slope) and b(intercept) of linear regression line
        m, b = np.polyfit(x, y, 1)
        ## add linear regression line to scatterplot 
        #plt.plot(x, m*x+b, color = 'red')
        ## add y = x line
        plt.plot(x, x, color = 'red')


        from scipy.stats import pearsonr
        pearsonr(x,y)[0]
        
plt.show()

## Plotting the fitted vs true y predictions

In [ ]:
plt.rcParams['figure.figsize'] = [12, 3]
labels = models[:]
labels.insert(0, 'none')

fig = plt.figure()
iter = 0

for pivot_DGP in range(-1,0):
        
    ## Get the data
    data_pivot = res_dict['pDGP: ' + str(pivot_DGP) + '; pfit: -1'][5]
    
    ## Get the true u values
    u_true = res_dict['pDGP: ' + str(pivot_DGP) + '; pfit: -1'][4]
    CAR_df = pd.DataFrame(u_true[0,:,:], columns = models)
    y_exp = tf.reduce_sum(data_pivot[models].values*u_true, axis = 2)[0].numpy()
    
    for pivot_fit in range(-1,2):
        iter = iter + 1
        key = 'pDGP: ' + str(pivot_DGP) + '; pfit: ' + str(pivot_fit)    

        ## get the ensemble weights
        phi = res_dict[key][0][0]
        
        ## Get the u values from each run
        predictions = np.empty(shape=(phi.shape[0], 
                              phi.shape[2]), 
                       dtype='float64')
        for i in range(phi.shape[2]):
            u = phi_to_u(phi.numpy()[:,:,i], pivot = pivot_fit)   
            predictions[:,i] = tf.reduce_sum(data_pivot[models].values*u, axis = 1)
    
        y_pred = np.mean(predictions, axis = 1)
            
        #CAR_df = pd.DataFrame(np.mean(u, axis = 2), columns = models)

        #plt.subplot(3, 4, iter)
        plt.subplot(1, 3, iter)

        x = data_pivot['census']
        y = y_pred
        #y = y_pred.numpy()
        plt.scatter(x, y)
        plt.xlabel("true census values")
        plt.ylabel(labels[pivot_fit + 1])

        #obtain m (slope) and b(intercept) of linear regression line
        m, b = np.polyfit(x, y.astype('float64'), 1)

        plt.xlim([min([min(x), min(y)]), max([max(x), max(y)])])
        plt.ylim([min([min(x), min(y)]), max([max(x), max(y)])])
        ax = plt.gca()
        ax.set_aspect('equal', adjustable='box')
        ax.set_xscale("log")
        ax.set_yscale("log")

        ## obtain m (slope) and b(intercept) of linear regression line
        m, b = np.polyfit(x, y, 1)
        ## add linear regression line to scatterplot 
        #plt.plot(x, m*x+b, color = 'red')
        ## add y = x line
        plt.plot(x, x, color = 'red')
        
plt.show()

## Plotting the chloroploth maps

In [ ]:
# Create the weights dict for plotting the outcomes

weights_dict = {
    "acs": CAR_ensemble_weights[:,0],
    "pep": CAR_ensemble_weights[:,1],
    "worldpop": CAR_ensemble_weights[:,2]
}

color_weights = make_color_norm(
    list(weights_dict.values())[1],   
    method="percentile")

In [ ]:
norm_weights_dict = {
    "acs": u[:,0],
    "pep": u[:,1],
    "worldpop": u[:,2]
}

color_norm_weights = make_color_norm(
    list(norm_weights_dict.values())[1],   
    method="percentile")

In [ ]:
import plotly
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
# df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
#                    dtype={"fips": str})
import matplotlib.colors as colors

import plotly.express as px

In [ ]:
for model_name in models:
    output = pd.DataFrame(np.column_stack([data_sub[["GEOID"]], weights_dict[model_name]]))
    output = output.set_axis(['GEOID', model_name], axis=1)
    output[model_name] = output[model_name].astype(float)
    fig = px.choropleth_mapbox(output, geojson=counties, locations='GEOID', color=model_name,
                           color_continuous_scale="Viridis",
                           #range_color=(0.05,0.07),
                           mapbox_style="carto-positron",
                           #mapbox_style='white-bg',
                           #featureidkey="properties.MWS_ID",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5#,
                           #labels={'unemp':'unemployment rate'}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
# pd.concat([data_NY[["GEOID"]].reset_index(drop=True), pd.DataFrame( phi_true[0].numpy(), columns = models)], axis=1)
# pd.DataFrame( phi_true[0].numpy(), columns = models)

In [ ]:
full_phi = pd.concat([data_sub[["GEOID"]].reset_index(drop=True), 
                         pd.DataFrame(phi_true[0].numpy(), columns = models)], 
                        axis=1)

full_u = pd.concat([data_sub[["GEOID"]].reset_index(drop=True), 
                         pd.DataFrame(u_true[0].numpy(), columns = models)], 
                        axis=1)


#full_phi[['GEOID', 'acs']], full_u

In [ ]:
for model_name in models:
    output = full_phi[['GEOID', model_name]]
    fig = px.choropleth_mapbox(output, geojson=counties, locations='GEOID', color=model_name,
                           color_continuous_scale="Viridis",
                           #range_color=(0.05,0.07),
                           mapbox_style="carto-positron",
                           #mapbox_style='white-bg',
                           #featureidkey="properties.MWS_ID",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5#,
                           #labels={'unemp':'unemployment rate'}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()